In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [37]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  GridSearchCV,KFold
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import train_test_split

In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [4]:
data=pd.read_csv("diabetes.csv")

In [5]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [6]:
data.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
Pregnancies                 768 non-null int64
Glucose                     768 non-null int64
BloodPressure               768 non-null int64
SkinThickness               768 non-null int64
Insulin                     768 non-null int64
BMI                         768 non-null float64
DiabetesPedigreeFunction    768 non-null float64
Age                         768 non-null int64
Outcome                     768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [8]:
# Minimum of glucose,BMI,BP shows out to be 0 (missing data/outlier)
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [9]:
data[data["Glucose"]==0]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
75,1,0,48,20,0,24.7,0.140,22,0
182,1,0,74,20,23,27.7,0.299,21,0
342,1,0,68,35,0,32.0,0.389,22,0
349,5,0,80,32,0,41.0,0.346,37,1
502,6,0,68,41,0,39.0,0.727,41,1


In [10]:
data[data["BMI"]==0]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
9,8,125,96,0,0,0.0,0.232,54,1
49,7,105,0,0,0,0.0,0.305,24,0
60,2,84,0,0,0,0.0,0.304,21,0
81,2,74,0,0,0,0.0,0.102,22,0
145,0,102,75,23,0,0.0,0.572,21,0
371,0,118,64,23,89,0.0,1.731,21,0
426,0,94,0,0,0,0.0,0.256,25,0
494,3,80,0,0,0,0.0,0.174,22,0
522,6,114,0,0,0,0.0,0.189,26,0
684,5,136,82,0,0,0.0,0.640,69,0


In [11]:
print("Shape of Dataset: ",data.shape)

Shape of Dataset:  (768, 9)


# Preprocess Dataset

In [12]:
# Replace 0 for these columns by numpy Nan
cols=["Glucose","BloodPressure","SkinThickness","Insulin","BMI"]

for c in cols:
    data[c].replace(0,np.NaN,inplace=True)

In [13]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,763.000000,733.000000,541.000000,394.000000,757.000000,768.000000,768.000000,768.000000
mean,3.845052,121.686763,72.405184,29.153420,155.548223,32.457464,0.471876,33.240885,0.348958
std,3.369578,30.535641,12.382158,10.476982,118.775855,6.924988,0.331329,11.760232,0.476951
min,0.000000,44.000000,24.000000,7.000000,14.000000,18.200000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,64.000000,22.000000,76.250000,27.500000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,29.000000,125.000000,32.300000,0.372500,29.000000,0.000000
75%,6.000000,141.000000,80.000000,36.000000,190.000000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [14]:
data.isnull().sum()

Pregnancies                   0
Glucose                       5
BloodPressure                35
SkinThickness               227
Insulin                     374
BMI                          11
DiabetesPedigreeFunction      0
Age                           0
Outcome                       0
dtype: int64

In [15]:
# Remove Null Values
data.dropna(inplace=True)
print("Shape of Dataset: ",data.shape)

Shape of Dataset:  (392, 9)


In [16]:
data.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000,392.000000
mean,3.301020,122.627551,70.663265,29.145408,156.056122,33.086224,0.523046,30.864796,0.331633
std,3.211424,30.860781,12.496092,10.516424,118.841690,7.027659,0.345488,10.200777,0.471401
min,0.000000,56.000000,24.000000,7.000000,14.000000,18.200000,0.085000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,21.000000,76.750000,28.400000,0.269750,23.000000,0.000000
50%,2.000000,119.000000,70.000000,29.000000,125.500000,33.200000,0.449500,27.000000,0.000000
75%,5.000000,143.000000,78.000000,37.000000,190.000000,37.100000,0.687000,36.000000,1.000000
max,17.000000,198.000000,110.000000,63.000000,846.000000,67.100000,2.420000,81.000000,1.000000


# Splitting Dataset

In [17]:
X=data.drop(["Outcome"],axis=1)
y=data["Outcome"]

In [18]:
print("Shape of X:",X.shape)
print("Shape of y:",y.shape)

Shape of X: (392, 8)
Shape of y: (392,)


In [19]:
X.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33
6,3,78.0,50.0,32.0,88.0,31.0,0.248,26
8,2,197.0,70.0,45.0,543.0,30.5,0.158,53
13,1,189.0,60.0,23.0,846.0,30.1,0.398,59


# Normalise the Scale

In [20]:
scaler=StandardScaler().fit(X)

In [21]:
X_std=scaler.transform(X)

Xfin=pd.DataFrame(X_std)

Xfin.describe()

,0,1,2,3,4,5,6,7
count,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02,3.920000e+02
mean,-4.021726e-17,3.129583e-17,-4.641624e-16,1.042250e-16,6.485742e-17,1.543550e-16,3.880116e-17,1.028089e-16
std,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00,1.001278e+00
min,-1.029213e+00,-2.161731e+00,-3.739001e+00,-2.108484e+00,-1.196867e+00,-2.120941e+00,-1.269525e+00,-9.682991e-01
25%,-7.174265e-01,-7.665958e-01,-6.941640e-01,-7.755315e-01,-6.681786e-01,-6.676780e-01,-7.340909e-01,-7.719850e-01
50%,-4.056403e-01,-1.176959e-01,-5.314565e-02,-1.384444e-02,-2.574448e-01,1.621036e-02,-2.131475e-01,-3.793569e-01
75%,5.297185e-01,6.609841e-01,5.878727e-01,7.478426e-01,2.859877e-01,5.718696e-01,4.751644e-01,5.040564e-01
max,4.271153e+00,2.445459e+00,3.151946e+00,3.223325e+00,5.812990e+00,4.846172e+00,5.497667e+00,4.921123e+00


In [41]:
Xfin.head()

,0,1,2,3,4,5,6,7
0,-0.717427,-1.091046,-0.373655,-0.585110,-0.522842,-0.710421,-1.031876,-0.968299
1,-1.029213,0.466314,-2.456964,0.557421,0.100631,1.426730,5.115111,0.209585
2,-0.093854,-1.447941,-1.655691,0.271788,-0.573394,-0.297238,-0.797126,-0.477514
3,-0.405640,2.413014,-0.053146,1.509530,3.260122,-0.368477,-1.057960,2.172726
4,-0.717427,2.153454,-0.854419,-0.585110,5.812990,-0.425468,-0.362402,2.761668


In [22]:
Xfin.columns

RangeIndex(start=0, stop=8, step=1)

In [23]:
# function to define Model
def  model_create():
    #Create Model
    model=Sequential()
    model.add(Dense(8,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(4,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    #Compile The Model
    adam = Adam(lr =0.01)
    model.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
    return model

In [24]:
model=model_create()
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 5         
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
None


# GridSearch Algorithm

In [25]:
# function to define Model
seed=6
np.random.seed(seed)
def  model_create_new():
    #Create Model
    model=Sequential()
    model.add(Dense(8,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(4,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    #Compile The Model
    adam = Adam(lr =0.01)
    model.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
    return model

model_new= KerasClassifier(build_fn= model_create_new,verbose=1)

# Define Grid Search

batch_size=[10,20,40]
epochs=[10,25,50]

# make a dictionary for gridSearch
param_grid=dict(batch_size=batch_size,epochs=epochs)

#Build The Grid Search
grid=GridSearchCV(estimator=model_new,param_grid=param_grid,cv=KFold(random_state=seed),verbose=5)

grid_results=grid.fit(Xfin,y)

# Summarise Results

print("Best Score {} , using {} ".format(grid_results.best_score_,grid_results.best_params_))
mean=grid_results.cv_results_['mean_test_score']
stds=grid_results.cv_results_['std_test_score']
params=grid_results.cv_results_['params']

for mean,stdev,params in zip(mean,stds,params):
    print("{} , {} with : {} ".format(mean,stdev,params))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] batch_size=10, epochs=10 ........................................


C:\Users\Dipto\Anaconda33\lib\site-packages\sklearn\model_selection\_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Instructions for updating:
Use tf.cast instead.
Epoch 1/10
261/261 [==============================] - 5s 20ms/step - loss: 0.6263 - acc: 0.6935
Epoch 2/10
261/261 [==============================] - 0s 278us/step - loss: 0.4782 - acc: 0.6973
Epoch 3/10
261/261 [==============================] - 0s 354us/step - loss: 0.4373 - acc: 0.6973
Epoch 4/10
261/261 [==============================] - 0s 332us/step - loss: 0.4240 - acc: 0.7011
Epoch 5/10
261/261 [==============================] - 0s 266us/step - loss: 0.4130 - acc: 0.8008
Epoch 6/10
261/261 [==============================] - 0s 227us/step - loss: 0.4001 - acc: 0.8352
Epoch 7/10
261/261 [==============================] - 0s 205us/step - loss: 0.3900 - acc: 0.8391
Epoch 8/10
261/261 [==============================] - 0s 183us/step - loss: 0.3877 - acc: 0.8238
Epoch 9/10
261/261 [==============================] - 0s 183us/step - loss: 0.3906 - acc: 0.8391
Epoch 10/10
131/131 [==============================] - 0s 2ms/step
[CV] ........

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.8s remaining:    0.0s


Epoch 1/10
261/261 [==============================] - 1s 5ms/step - loss: 0.6398 - acc: 0.6475
Epoch 2/10
261/261 [==============================] - 0s 189us/step - loss: 0.5226 - acc: 0.6552
Epoch 3/10
261/261 [==============================] - 0s 236us/step - loss: 0.4920 - acc: 0.6552
Epoch 4/10
261/261 [==============================] - 0s 194us/step - loss: 0.4815 - acc: 0.7011
Epoch 5/10
261/261 [==============================] - 0s 199us/step - loss: 0.4743 - acc: 0.7893
Epoch 6/10
261/261 [==============================] - 0s 192us/step - loss: 0.4616 - acc: 0.8084
Epoch 7/10
261/261 [==============================] - ETA: 0s - loss: 0.4551 - acc: 0.800 - 0s 207us/step - loss: 0.4546 - acc: 0.8008
Epoch 8/10
261/261 [==============================] - 0s 221us/step - loss: 0.4464 - acc: 0.8046
Epoch 9/10
261/261 [==============================] - 0s 200us/step - loss: 0.4344 - acc: 0.8046
Epoch 10/10
131/131 [==============================] - 0s 2ms/step
[CV] ............ batch_

...

262/262 [==============================] - 0s 88us/step - loss: 0.5369 - acc: 0.7634
Epoch 14/25
262/262 [==============================] - 0s 94us/step - loss: 0.5264 - acc: 0.7824
Epoch 15/25
262/262 [==============================] - 0s 89us/step - loss: 0.5216 - acc: 0.7977
Epoch 16/25
262/262 [==============================] - 0s 125us/step - loss: 0.5167 - acc: 0.7824
Epoch 17/25
262/262 [==============================] - 0s 104us/step - loss: 0.5060 - acc: 0.7824
Epoch 18/25
262/262 [==============================] - 0s 108us/step - loss: 0.5021 - acc: 0.7901
Epoch 19/25
262/262 [==============================] - 0s 92us/step - loss: 0.4960 - acc: 0.7863
Epoch 20/25
262/262 [==============================] - 0s 99us/step - loss: 0.4905 - acc: 0.7863
Epoch 21/25
262/262 [==============================] - 0s 79us/step - loss: 0.4851 - acc: 0.7977
Epoch 22/25
262/262 [==============================] - 0s 84us/step - loss: 0.4810 - acc: 0.7977
Epoch 23/25
262/262 [==================

261/261 [==============================] - 0s 107us/step - loss: 0.4052 - acc: 0.8238
Epoch 19/50
261/261 [==============================] - 0s 112us/step - loss: 0.4025 - acc: 0.8199
Epoch 20/50
261/261 [==============================] - 0s 97us/step - loss: 0.4017 - acc: 0.8199
Epoch 21/50
261/261 [==============================] - 0s 80us/step - loss: 0.3968 - acc: 0.8238
Epoch 22/50
261/261 [==============================] - 0s 101us/step - loss: 0.3965 - acc: 0.8199
Epoch 23/50
261/261 [==============================] - 0s 118us/step - loss: 0.3944 - acc: 0.8161
Epoch 24/50
261/261 [==============================] - 0s 90us/step - loss: 0.3979 - acc: 0.8238
Epoch 25/50
261/261 [==============================] - 0s 102us/step - loss: 0.3897 - acc: 0.8314
Epoch 26/50
261/261 [==============================] - 0s 122us/step - loss: 0.3882 - acc: 0.8199
Epoch 27/50
261/261 [==============================] - 0s 84us/step - loss: 0.3869 - acc: 0.8238
Epoch 28/50
261/261 [===============

130/130 [==============================] - 1s 6ms/step
[CV] ............ batch_size=40, epochs=50, score=0.854, total=   5.9s


[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  2.2min finished
C:\Users\Dipto\Anaconda33\lib\site-packages\sklearn\model_selection\_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Epoch 1/25
392/392 [==============================] - 3s 7ms/step - loss: 0.6706 - acc: 0.6556
Epoch 2/25
392/392 [==============================] - 0s 73us/step - loss: 0.5914 - acc: 0.7372
Epoch 3/25
392/392 [==============================] - 0s 76us/step - loss: 0.5012 - acc: 0.7704
Epoch 4/25
392/392 [==============================] - 0s 69us/step - loss: 0.4591 - acc: 0.7704
Epoch 5/25
392/392 [==============================] - 0s 81us/step - loss: 0.4522 - acc: 0.7653
Epoch 6/25
392/392 [==============================] - 0s 70us/step - loss: 0.4393 - acc: 0.7806
Epoch 7/25
392/392 [==============================] - 0s 77us/step - loss: 0.4354 - acc: 0.7985
Epoch 8/25
392/392 [==============================] - 0s 78us/step - loss: 0.4325 - acc: 0.7755
Epoch 9/25
392/392 [==============================] - 0s 74us/step - loss: 0.4278 - acc: 0.7806
Epoch 10/25
392/392 [==============================] - 0s 76us/step - loss: 0.4297 - acc: 0.7883
Epoch 11/25
392/392 [===================

# Droupot Rate and learning rate

In [27]:
from keras.layers import Dropout
#WE need to optimize Learning Rate
# function to define Model
seed=6
np.random.seed(seed)
def  model_create_new(learn_rate,dropout_rate):
    #Create Model
    model=Sequential()
    model.add(Dense(8,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dropout(dropout_rate))
    # Prevents a Neuron to become too important , helps model to generalise
    model.add(Dense(4,input_dim=8,kernel_initializer='normal',activation='relu'))
    model.add(Dropout(dropout_rate))
    # dropout_rate is between 0 and 1
    model.add(Dense(1,activation='sigmoid'))
    #Compile The Model
    adam = Adam(lr = learn_rate)
    model.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
    return model

model_new= KerasClassifier(build_fn= model_create_new,verbose=0,batch_size=40,epochs=25)
# Best batch size and epochs found earlier


# Define Grid Search

learn_rate=[0.1,0.01,0.001]
# Influences magnitude of change of parameters
dropout_rate=[0.0,0.1,0.2]

# make a dictionary for gridSearch
param_grid=dict(learn_rate=learn_rate,dropout_rate=dropout_rate)

#Build The Grid Search
grid=GridSearchCV(estimator=model_new,param_grid=param_grid,cv=KFold(random_state=seed),verbose=5)

grid_results=grid.fit(Xfin,y)

# Summarise Results

print("Best Score {} , using {} ".format(grid_results.best_score_,grid_results.best_params_))
mean=grid_results.cv_results_['mean_test_score']
stds=grid_results.cv_results_['std_test_score']
params=grid_results.cv_results_['params']

for mean,stdev,params in zip(mean,stds,params):
    print("{} , {} with : {} ".format(mean,stdev,params))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] dropout_rate=0.0, learn_rate=0.1 ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.771, total=   5.8s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    5.7s remaining:    0.0s


[CV] .... dropout_rate=0.0, learn_rate=0.1, score=0.756, total=   5.9s
[CV] dropout_rate=0.0, learn_rate=0.1 ................................


...

[CV] ... dropout_rate=0.0, learn_rate=0.01, score=0.756, total=   6.1s
[CV] dropout_rate=0.0, learn_rate=0.01 ...............................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   24.1s remaining:    0.0s


[CV] ... dropout_rate=0.0, learn_rate=0.01, score=0.771, total=   5.9s
[CV] dropout_rate=0.0, learn_rate=0.01 ...............................
[CV] ... dropout_rate=0.0, learn_rate=0.01, score=0.862, total=   6.0s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.733, total=   7.2s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.748, total=   6.4s
[CV] dropout_rate=0.0, learn_rate=0.001 ..............................
[CV] .. dropout_rate=0.0, learn_rate=0.001, score=0.700, total=   6.6s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
[CV] .... dropout_rate=0.1, learn_rate=0.1, score=0.718, total=   8.4s
[CV] dropout_rate=0.1, learn_rate=0.1 ................................
[CV] .... dropout_rate=0.1, learn

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  3.7min finished


Best Score 0.7959183647620435 , using {'dropout_rate': 0.0, 'learn_rate': 0.01} 
0.7780612202323213 , 0.02178523584090544 with : {'dropout_rate': 0.0, 'learn_rate': 0.1} 
0.7959183647620435 , 0.04664236281877595 with : {'dropout_rate': 0.0, 'learn_rate': 0.01} 
0.7270408046184754 , 0.020043924183272166 with : {'dropout_rate': 0.0, 'learn_rate': 0.001} 
0.7704081591598841 , 0.0430524149010169 with : {'dropout_rate': 0.1, 'learn_rate': 0.1} 
0.77551020392958 , 0.03942172171987488 with : {'dropout_rate': 0.1, 'learn_rate': 0.01} 
0.7704081571831995 , 0.021758740536289154 with : {'dropout_rate': 0.1, 'learn_rate': 0.001} 
0.7499999940699461 , 0.02739208015106309 with : {'dropout_rate': 0.2, 'learn_rate': 0.1} 
0.7729591880829967 , 0.03530307839688959 with : {'dropout_rate': 0.2, 'learn_rate': 0.01} 
0.7755102001282633 , 0.024525924203499666 with : {'dropout_rate': 0.2, 'learn_rate': 0.001} 


# Kernel Initialization and Activation

In [28]:
# function to define Model
seed=6
np.random.seed(seed)
def  model_create_new(activation,init):
    #Create Model
    model=Sequential()
    model.add(Dense(8,input_dim=8,kernel_initializer=init,activation=activation))
    model.add(Dense(4,input_dim=8,kernel_initializer=init,activation=activation))
    model.add(Dense(1,activation='sigmoid'))
    #Compile The Model
    adam = Adam(lr =0.01)
    model.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
    return model

model_new= KerasClassifier(build_fn= model_create_new,verbose=0,batch_size=40,epochs=25)
# Define Grid Search

# Optimize the activation function and initializer
activation=["softmax","relu","tanh","linear"]
init=["uniform","normal","zero"]

# make a dictionary for gridSearch
param_grid=dict(activation=activation,init =init)

#Build The Grid Search
grid=GridSearchCV(estimator=model_new,param_grid=param_grid,cv=KFold(random_state=seed),verbose=5)

grid_results=grid.fit(Xfin,y)

# Summarise Results

print("Best Score {} , using {} ".format(grid_results.best_score_,grid_results.best_params_))
mean=grid_results.cv_results_['mean_test_score']
stds=grid_results.cv_results_['std_test_score']
params=grid_results.cv_results_['params']

for mean,stdev,params in zip(mean,stds,params):
    print("{} , {} with : {} ".format(mean,stdev,params))

C:\Users\Dipto\Anaconda33\lib\site-packages\sklearn\model_selection\_split.py:431: FutureWarning: The default value of n_split will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] activation=softmax, init=uniform ................................
[CV] .... activation=softmax, init=uniform, score=0.695, total=  10.2s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   10.1s remaining:    0.0s


[CV] .... activation=softmax, init=uniform, score=0.763, total=  10.1s
[CV] activation=softmax, init=uniform ................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   20.3s remaining:    0.0s


...

[CV] ..... activation=softmax, init=normal, score=0.710, total=  10.9s
[CV] activation=softmax, init=normal .................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   41.8s remaining:    0.0s


[CV] ..... activation=softmax, init=normal, score=0.771, total=  11.2s
[CV] activation=softmax, init=normal .................................
[CV] ..... activation=softmax, init=normal, score=0.808, total=  10.8s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.611, total=  11.4s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.695, total=  11.6s
[CV] activation=softmax, init=zero ...................................
[CV] ....... activation=softmax, init=zero, score=0.700, total=  10.9s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.733, total=  11.1s
[CV] activation=relu, init=uniform ...................................
[CV] ....... activation=relu, init=uniform, score=0.748, total=  10.6s
[CV] activation=relu, init=uniform ...................................
[CV] .

[Parallel(n_jobs=1)]: Done  36 out of  36 | elapsed:  7.6min finished


Best Score 0.7908163260744543 , using {'activation': 'linear', 'init': 'normal'} 
0.7525510166068466 , 0.04365529000660255 with : {'activation': 'softmax', 'init': 'uniform'} 
0.7627551006723423 , 0.040312763309779176 with : {'activation': 'softmax', 'init': 'normal'} 
0.6683673428333535 , 0.04092231505753368 with : {'activation': 'softmax', 'init': 'zero'} 
0.7729591847378381 , 0.04656018682914353 with : {'activation': 'relu', 'init': 'uniform'} 
0.7755102054501066 , 0.04319314111244652 with : {'activation': 'relu', 'init': 'normal'} 
0.6683673428333535 , 0.04092231505753368 with : {'activation': 'relu', 'init': 'zero'} 
0.7882653141812402 , 0.041837549762974366 with : {'activation': 'tanh', 'init': 'uniform'} 
0.7831632645458592 , 0.04090311953983408 with : {'activation': 'tanh', 'init': 'normal'} 
0.6683673428333535 , 0.04092231505753368 with : {'activation': 'tanh', 'init': 'zero'} 
0.7882653080991336 , 0.03010204819047102 with : {'activation': 'linear', 'init': 'uniform'} 
0.79081

# No of Neurons

In [29]:
# function to define Model
seed=6
np.random.seed(seed)
def  model_create_new(neuron1,neuron2):
    #Create Model
    model=Sequential()
    model.add(Dense(neuron1,input_dim=8,kernel_initializer='normal',activation='linear'))
    model.add(Dense(neuron2,input_dim=neuron1,kernel_initializer='normal',activation='linear'))
    model.add(Dense(1,activation='sigmoid'))
    #Compile The Model
    adam = Adam(lr =0.01)
    model.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
    return model

model_new= KerasClassifier(build_fn= model_create_new,verbose=0,batch_size=40,epochs=25)
# Define Grid Search

# Optimize the neurons 
neuron1=[4,8,16]
neuron2=[2,4,8]

# make a dictionary for gridSearch
param_grid=dict(neuron1=neuron1,neuron2=neuron2)

#Build The Grid Search
grid=GridSearchCV(estimator=model_new,param_grid=param_grid,cv=KFold(random_state=seed),verbose=5)

grid_results=grid.fit(Xfin,y)

# Summarise Results

print("Best Score {} , using {} ".format(grid_results.best_score_,grid_results.best_params_))
mean=grid_results.cv_results_['mean_test_score']
stds=grid_results.cv_results_['std_test_score']
params=grid_results.cv_results_['params']

for mean,stdev,params in zip(mean,stds,params):
    print("{} , {} with : {} ".format(mean,stdev,params))

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] ................ neuron1=4, neuron2=2, score=0.763, total=  15.6s
[CV] neuron1=4, neuron2=2 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.5s remaining:    0.0s


[CV] ................ neuron1=4, neuron2=2, score=0.763, total=  16.1s
[CV] neuron1=4, neuron2=2 ............................................


...

[CV] ................ neuron1=4, neuron2=4, score=0.763, total=  14.7s
[CV] neuron1=4, neuron2=4 ............................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  1.0min remaining:    0.0s


[CV] ................ neuron1=4, neuron2=4, score=0.763, total=  14.7s
[CV] neuron1=4, neuron2=4 ............................................
[CV] ................ neuron1=4, neuron2=4, score=0.838, total=  16.8s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.771, total=  17.8s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.771, total=  15.9s
[CV] neuron1=4, neuron2=8 ............................................
[CV] ................ neuron1=4, neuron2=8, score=0.815, total=  17.4s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ................ neuron1=8, neuron2=2, score=0.763, total=  18.1s
[CV] neuron1=8, neuron2=2 ............................................
[CV] ................ neuron1=8, neuron2=2, score=0.763, total=  18.0s
[CV] neuron1=8, neuron2=2 ............................................
[CV] .

[Parallel(n_jobs=1)]: Done  27 out of  27 | elapsed:  7.9min finished


Best Score 0.7933673440497748 , using {'neuron1': 16, 'neuron2': 2} 
0.7831632660663858 , 0.028115285089042093 with : {'neuron1': 4, 'neuron2': 2} 
0.7882653080991336 , 0.035358360413807126 with : {'neuron1': 4, 'neuron2': 4} 
0.7857142886032864 , 0.0208998572485492 with : {'neuron1': 4, 'neuron2': 8} 
0.7831632645458592 , 0.02811529584910769 with : {'neuron1': 8, 'neuron2': 2} 
0.7908163260744543 , 0.03370616232395781 with : {'neuron1': 8, 'neuron2': 4} 
0.7882653080991336 , 0.03010204819047102 with : {'neuron1': 8, 'neuron2': 8} 
0.7933673440497748 , 0.03731364977453912 with : {'neuron1': 16, 'neuron2': 2} 
0.7831632599842792 , 0.028115289373295223 with : {'neuron1': 16, 'neuron2': 4} 
0.7908163260744543 , 0.03370616232395781 with : {'neuron1': 16, 'neuron2': 8} 


In [30]:
# Generate Optimal Predictions
y_pred=grid.predict(Xfin)

In [32]:
# Check Accuracy
print(accuracy_score(y,y_pred))

print(classification_report(y,y_pred))

0.7857142857142857
              precision    recall  f1-score   support

           0       0.81      0.89      0.85       262
           1       0.73      0.57      0.64       130

    accuracy                           0.79       392
   macro avg       0.77      0.73      0.74       392
weighted avg       0.78      0.79      0.78       392



In [51]:
# function to define Model
seed=6
np.random.seed(seed)
def  model_create_new():
    #Create Model
    model=Sequential()
    model.add(Dense(16,input_dim=8,kernel_initializer='normal',activation='linear'))
    model.add(Dense(2,input_dim=16,kernel_initializer='normal',activation='linear'))
    model.add(Dense(1,activation='sigmoid'))
    #Compile The Model
    adam = Adam(lr =0.01)
    model.compile(loss="binary_crossentropy",optimizer=adam,metrics=['accuracy'])
    return model

final_model=model_create_new()

In [42]:
X_train, X_test, y_train, y_test = train_test_split(Xfin, y, test_size=0.33, random_state=42)

In [45]:
print("Shape of X:",Xfin.shape)
print("Shape of y",y.shape)
print("Shape of X train:",X_train.shape)
print("Shape of y train",y_train.shape)
print("Shape of X test:",X_test.shape)
print("Shape of y test",y_test.shape)


Shape of X: (392, 8)
Shape of y (392,)
Shape of X train: (262, 8)
Shape of y train (262,)
Shape of X test: (130, 8)
Shape of y test (130,)


In [52]:
# Fit the model
final_model.fit(X_train,y_train,batch_size=40,epochs=25)

Epoch 1/25
262/262 [==============================] - 14s 52ms/step - loss: 0.6217 - acc: 0.7595
Epoch 2/25
262/262 [==============================] - 0s 239us/step - loss: 0.4743 - acc: 0.8130
Epoch 3/25
262/262 [==============================] - 0s 200us/step - loss: 0.4356 - acc: 0.7863
Epoch 4/25
262/262 [==============================] - 0s 199us/step - loss: 0.4307 - acc: 0.7748
Epoch 5/25
262/262 [==============================] - 0s 207us/step - loss: 0.4200 - acc: 0.7939
Epoch 6/25
262/262 [==============================] - 0s 212us/step - loss: 0.4160 - acc: 0.7863
Epoch 7/25
262/262 [==============================] - 0s 220us/step - loss: 0.4146 - acc: 0.7939
Epoch 8/25
262/262 [==============================] - 0s 199us/step - loss: 0.4149 - acc: 0.8015
Epoch 9/25
262/262 [==============================] - 0s 180us/step - loss: 0.4132 - acc: 0.8015
Epoch 10/25
262/262 [==============================] - 0s 208us/step - loss: 0.4151 - acc: 0.8015
Epoch 11/25
262/262 [========

In [53]:
# Predict from testing set
preds= final_model.predict(X_test)

In [55]:
preds=(preds > 0.5)

In [56]:
# Check Accuracy
print(accuracy_score(y_test,preds))

print(classification_report(y_test,preds))

0.7384615384615385
              precision    recall  f1-score   support

           0       0.79      0.84      0.82        89
           1       0.60      0.51      0.55        41

    accuracy                           0.74       130
   macro avg       0.69      0.68      0.68       130
weighted avg       0.73      0.74      0.73       130

